In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [9]:
train_dir = "data/train"
test_dir  = "data/test"

device = "cuda" if torch.cuda.is_available() else "cpu"
num_epochs = 60
batch_size = 128
learning_rate = 1e-4

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=test_dir,  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)
print("Number of classes:", num_classes)


Classes: ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot']
Number of classes: 28


In [11]:
class PlantDiseaseCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlantDiseaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool2d(kernel_size=2, stride=2)
        # After pooling once on 224x224 => shape is [32, 112, 112]
        self.fc1   = nn.Linear(32 * 112 * 112, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)             # from 224->112
        x = x.view(x.size(0), -1)    # flatten
        x = self.fc1(x)              # logits
        return x

model = PlantDiseaseCNN(num_classes).to(device)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=30, device='cpu'):
    for epoch in range(num_epochs):
        # ---- TRAIN PHASE ----
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            train_total   += labels.size(0)

        epoch_train_loss = train_loss / train_total
        epoch_train_acc  = 100.0 * train_correct / train_total

        # ---- TEST PHASE ----
        model.eval()
        test_loss, test_correct, test_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == labels).sum().item()
                test_total   += labels.size(0)

        epoch_test_loss = test_loss / test_total
        epoch_test_acc  = 100.0 * test_correct / test_total

        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}% || "
              f"Test Loss: {epoch_test_loss:.4f} | Test Acc: {epoch_test_acc:.2f}%")


In [14]:
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=num_epochs, device=device)

Epoch [1/60] Train Loss: 15.1442 | Train Acc: 5.86% || Test Loss: 18.3965 | Test Acc: 5.93%
Epoch [2/60] Train Loss: 7.8532 | Train Acc: 10.36% || Test Loss: 10.0662 | Test Acc: 8.47%
Epoch [3/60] Train Loss: 4.5618 | Train Acc: 15.54% || Test Loss: 7.2577 | Test Acc: 8.05%
Epoch [4/60] Train Loss: 3.5152 | Train Acc: 20.08% || Test Loss: 6.4388 | Test Acc: 11.86%
Epoch [5/60] Train Loss: 3.2432 | Train Acc: 21.53% || Test Loss: 6.8819 | Test Acc: 7.20%
Epoch [6/60] Train Loss: 3.2982 | Train Acc: 23.42% || Test Loss: 6.0098 | Test Acc: 9.32%
Epoch [7/60] Train Loss: 3.0831 | Train Acc: 26.11% || Test Loss: 6.7500 | Test Acc: 8.90%
Epoch [8/60] Train Loss: 3.1631 | Train Acc: 25.77% || Test Loss: 6.1886 | Test Acc: 8.05%
Epoch [9/60] Train Loss: 2.9772 | Train Acc: 27.83% || Test Loss: 5.4683 | Test Acc: 12.71%
Epoch [10/60] Train Loss: 2.7040 | Train Acc: 30.95% || Test Loss: 5.8500 | Test Acc: 12.29%
Epoch [11/60] Train Loss: 2.8378 | Train Acc: 31.38% || Test Loss: 7.0883 | Test Acc